<a href="https://colab.research.google.com/github/shriraoshiwani11/Splitwise/blob/main/Splitwise_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from typing import List, Optional, Union
from collections import defaultdict, ChainMap
from enum import Enum

#An enumeration class that defines three types of expense
class SplitType(Enum):
  EQUAL = 'equal'
  EXACT = 'exact'
  PERCENT = 'percent'

#An enumeration class that defines two types of transactions
class TransactionType(Enum):
  OWE = 'owe'
  LEND = 'lend'


#A class that represents a transaction, amount, user ID, and transaction type
class Transaction:
  def __init__(self, amount: int, user_id: str, type: TransactionType):
    self.amount = amount
    self._type = type
    self.user_id = user_id

  @property
  def owed(self) -> bool:
    return self._type == TransactionType.OWE

  @property
  def lent(self) -> bool:
    return self._type == TransactionType.LEND


 #The main class that manages expenses and balances.
 #It provides methods to record expenses, validate inputs, calculate balances, and display balances.
class SplitService:
  def __init__(self):
    self.transactions_for_users = defaultdict(list)


#This method validates the inputs.
#calculates the transactions between users based on the split type.
  def expense(self, amount_paid: int, user_owed: str, num_users: int, users: List[str], split_type: SplitType, split_amount: Optional[List[Union[int, float]]] = None):
    self.validate(split_type=split_type, split_amount=split_amount, num_users=num_users, amount_paid=amount_paid)

#Spliting the expense  equally among all users.
    if split_type == SplitType.EQUAL:
      amount_owed = amount_paid / num_users

      for user_id in users:
        if user_id == user_owed:
          continue

        self.transactions_for_users[user_owed].append(Transaction(user_id=user_id, amount=amount_owed, type=TransactionType.LEND))
        self.transactions_for_users[user_id].append(Transaction(user_id=user_owed, amount=amount_owed, type=TransactionType.OWE))

#Spliting the expense based on specific amounts provided for each user.
    if split_type == SplitType.EXACT:
      for user_id, amount_owed in zip(users, split_amount):
        if user_id == user_owed:
          continue

        self.transactions_for_users[user_owed].append(Transaction(user_id=user_id, amount=amount_owed, type=TransactionType.LEND))
        self.transactions_for_users[user_id].append(Transaction(user_id=user_owed, amount=amount_owed, type=TransactionType.OWE))


#Spliting the expense based on percentages assigned to each us
    if split_type == SplitType.PERCENT:
      for user_id, owed_percent in zip(users, split_amount):
        if user_id == user_owed:
          continue

        amount_owed = round((amount_paid * owed_percent / 100), 2)

        #self.transactions_for_users[user_owed].append(Transaction(user_id=user_id, amount=amount_owed, type=TransactionType.LEND))
        self.transactions_for_users[user_id].append(Transaction(user_id=user_owed, amount=amount_owed, type=TransactionType.OWE))


#Validates the inputs for the expense method based on the split type.
  def validate(self, split_type: SplitType, split_amount: List[int], num_users: int, amount_paid: int):
    if split_type == SplitType.EQUAL:
      return

    if split_type == SplitType.EXACT:
      if num_users != len(split_amount):
        raise Exception(f'The number of users owing {len(split_amount)}, does not equal the total number of users {num_users}')

      if amount_paid != sum(split_amount):
        raise Exception(f'The sum of the split amount {split_amount} = {sum(split_amount)} does not equal the total amount paid {amount_paid}')

    if split_type == SplitType.PERCENT:
      if num_users != len(split_amount):
        raise Exception(f'The number of users owing {len(split_amount)}, does not equal the total number of users {num_users}')

      if 100 != sum(split_amount):
        raise Exception(f'The total percentage of {sum(split_amount)} does not equal 100')


#Calculating the balances for a specific user.
  def calculate_transactions(self, user_id: str):
    if user_id not in self.transactions_for_users:
      print(f'No balances for {user_id}')
      return {}

#Creating the transaction map and dictionary of user in debt.
    transaction_map = defaultdict(int)
    users_in_debt = defaultdict(list)

    for transaction in self.transactions_for_users[user_id]:
      if transaction.owed:
        transaction_map[transaction.user_id] += transaction.amount

      if transaction.lent:
        transaction_map[transaction.user_id] -= transaction.amount


    if all(amount_owed == 0 for _, amount_owed in transaction_map.items()):
      return {}

    for other_user_id, amount_owed in transaction_map.items():
      amount_owed = round(amount_owed, 2)

      if amount_owed < 0:
        users_in_debt[other_user_id].append((user_id, abs(amount_owed)))

      if amount_owed > 0:
        users_in_debt[user_id].append((other_user_id, amount_owed))

    return users_in_debt

#Displaying the balances for all users.
  def show(self, user_id: Optional[str] = None):
    users_in_debt = defaultdict(list)

    if user_id:
      print('====' * 10)
      print(f'showing transactions for user: {user_id}\n')

      users_in_debt = self.calculate_transactions(user_id=user_id)
    else:
      print('====' * 10)
      print('showing transactions for all\n')

      for user_id in self.transactions_for_users.keys():
        for user_in_debt, owed_users in self.calculate_transactions(user_id=user_id).items():
          users_in_debt[user_in_debt] = list(set(users_in_debt[user_in_debt] + owed_users))

    if not users_in_debt:
      print('No balances')

    for user_in_debt, users_owed in users_in_debt.items():
        for (user_owed, amount_owed) in users_owed:
          print(f'{user_in_debt} owes {user_owed}: {abs(amount_owed)}')

# Inputs
#Create an instance of the Splitservice class
split_service = SplitService()

#calling the show method to Display the balances for all users
split_service.show()

#Display balances for a specific user (u1)
split_service.show(user_id='u1')

#Problem statement 1
#Record an expense of $1000, split equally among 4 users (u1, u2, u3, u4)
split_service.expense(amount_paid=1000,
                      user_owed='u1',
                      num_users=4,
                      users=['u1', 'u2', 'u3', 'u4'],
                      split_type=SplitType.EQUAL)

#Display balances for a specific user (u4) and (u1)
split_service.show(user_id='u4')
split_service.show(user_id='u1')

#Problem statement 2
#Record an expense of $1250, split based on specific amounts for each user (u2: $370, u3: $880)
split_service.expense(amount_paid=1250,
                      user_owed='u1',
                      num_users=2,
                      users=['u2', 'u3'],
                      split_type=SplitType.EXACT,
                      split_amount=[370, 880])

# Display balances for all users
split_service.show()

#Problem statement 3
#Record an expense of $1200, split based on percentages for each user (u1: 40%, u2: 20%, u3: 20%, u4: 20%)
split_service.expense(amount_paid=1200,
                      user_owed='u4',
                      num_users=4,
                      users=['u1', 'u2', 'u3', 'u4'],
                      split_type=SplitType.PERCENT,
                      split_amount=[40, 20, 20, 20])

# Display balances for all users
split_service.show(user_id='u1')
split_service.show()

showing transactions for all

No balances
showing transactions for user: u1

No balances for u1
No balances
showing transactions for user: u4

u4 owes u1: 250.0
showing transactions for user: u1

u2 owes u1: 250.0
u3 owes u1: 250.0
u4 owes u1: 250.0
showing transactions for all

u2 owes u1: 620.0
u3 owes u1: 1130.0
u4 owes u1: 250.0
showing transactions for user: u1

u2 owes u1: 620.0
u3 owes u1: 1130.0
u1 owes u4: 230.0
showing transactions for all

u2 owes u4: 240.0
u2 owes u1: 620.0
u3 owes u1: 1130.0
u3 owes u4: 240.0
u1 owes u4: 230.0
u4 owes u1: 250.0
